In [ ]:
import sys
sys.path.append('..')
import backtrader as bt
import pandas as pd
import quantstats as qs
import matplotlib.pyplot as plt

# === 策略一：支援多空方向的 MACD 策略 ===
class MACDStrategy(bt.Strategy):
    def __init__(self):
        macd = bt.indicators.MACD(self.data.close)
        self.crossover = bt.indicators.CrossOver(macd.macd, macd.signal)
        self.trade_records = []
        self.long_order = None
        self.short_order = None

    def notify_order(self, order):
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                if order.parent is None:
                    print(f"[MACD LONG FILLED] Buy at {order.executed.price}, size: {order.executed.size}")
                else:
                    print(f"[MACD SHORT SL/TP] Buy at {order.executed.price}, size: {order.executed.size}")
                self.long_order = None
            elif order.issell():
                if order.parent is None:
                    print(f"[MACD SHORT FILLED] Sell at {order.executed.price}, size: {order.executed.size}")
                else:
                    print(f"[MACD LONG SL/TP] Sell at {order.executed.price}, size: {order.executed.size}")
                self.short_order = None

    def notify_trade(self, trade):
        if trade.isclosed:
            action = "[MACD LONG EXIT]" if trade.size > 0 else "[MACD SHORT EXIT]"
            reason = " (TP/SL)" if trade.commission > 0 else " (Manual)"
            print(f"{action}{reason} Exit at {self.data.close[0]} on {self.data.datetime.date(0).isoformat()}, PnL: {trade.pnl:.2f}")
            self.trade_records.append({
                'datetime': self.data.datetime.date(0).isoformat(),
                'entry_price': trade.price,
                'exit_price': self.data.close[0],
                'size': trade.size,
                'pnl': trade.pnl,
                'pnl_comm': trade.pnlcomm,
                'commission': trade.commission,
                'barlen': trade.barlen,
                'gross_pnl': trade.pnl,
                'net_pnl': trade.pnlcomm,
                'direction': 'Long' if trade.size > 0 else 'Short'
            })

# === 設定止損止盈 ===
    def next(self):
        dt = self.data.datetime.date(0).isoformat()
        price = self.data.close[0]

        if self.crossover > 0 and self.long_order is None:
            entry = self.buy(size=1, exectype=bt.Order.StopTrail,trailamount=0.25)

            self.long_order = entry
            print(f"[MACD LONG ENTRY] Buy at {price} on {dt}")

        # if self.crossover < 0 and self.short_order is None:
        #     entry = self.sell()
        #     stop_price = price + 500 / size
        #     limit_price = price - 1000 / size

        #     self.buy(exectype=bt.Order.Stop, price=stop_price, parent=entry)
        #     self.buy(exectype=bt.Order.Limit, price=limit_price, parent=entry)

        #     self.short_order = entry
        #     print(f"[MACD SHORT ENTRY] Sell at {price} on {dt}")

# === 策略二：RSI過熱策略 ===
class RSIStrategy(bt.Strategy):
    def __init__(self):
        self.rsi = bt.indicators.RSI(self.data.close, period=14)
        self.trade_records = []

    def notify_trade(self, trade):
        if trade.isclosed:
            print(f"[RSI EXIT] Exit at {self.data.close[0]} on {self.data.datetime.date(0).isoformat()}, PnL: {trade.pnl:.2f}")
            self.trade_records.append({
                'datetime': self.data.datetime.date(0).isoformat(),
                'entry_price': trade.price,
                'exit_price': self.data.close[0],
                'size': trade.size,
                'pnl': trade.pnl,
                'pnl_comm': trade.pnlcomm,
                'commission': trade.commission,
                'barlen': trade.barlen,
                'gross_pnl': trade.pnl,
                'net_pnl': trade.pnlcomm,
                'direction': 'Long' if trade.size > 0 else 'Short'
            })

    def next(self):
        dt = self.data.datetime.date(0).isoformat()
        price = self.data.close[0]

        if not self.position:
            if self.rsi < 30:
                #print(f"[RSI LONG ENTRY] Buy at {price} on {dt}")
                self.buy()
        elif self.rsi > 70:
            #print(f"[RSI EXIT SIGNAL] Close at {price} on {dt}")
            self.close()

# === 載入資料並篩選繪圖期間 ===
dataframe = pd.read_csv('/Users/coconut/Backtrade/datas/BTCUSDT_futures_4h_from_20210101.csv', index_col=0, parse_dates=True)
dataframe = dataframe[(dataframe.index >= '2021-01-01') & (dataframe.index <= '2024-12-31')]
data = bt.feeds.PandasData(dataname=dataframe)

# === 建立 Cerebro，加入兩個策略與分析器 ===
cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(MACDStrategy)
cerebro.addstrategy(RSIStrategy)
cerebro.broker.setcash(1000000)
cerebro.broker.setcommission(commission=0.002)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='ta')

# === 執行回測 ===
results = cerebro.run()

# === 擷取每筆交易記錄為 DataFrame ===
trades_macd = pd.DataFrame(results[0].trade_records)
trades_rsi = pd.DataFrame(results[1].trade_records)

trades_macd.to_csv("macd_trades.csv", index=False)
trades_rsi.to_csv("rsi_trades.csv", index=False)

print("MACD 策略交易記錄：")
print(trades_macd)
print("\nRSI 策略交易記錄：")
print(trades_rsi)

# === 合併並計算每日報酬 ===
all_trades = pd.concat([trades_macd, trades_rsi])
all_trades['datetime'] = pd.to_datetime(all_trades['datetime'])
all_trades.set_index('datetime', inplace=True)

# 損益彙總 → 報酬率
daily_returns = all_trades.groupby(all_trades.index.date)['net_pnl'].sum()
daily_returns = pd.Series(daily_returns)
daily_returns.index = pd.to_datetime(daily_returns.index)
daily_returns = daily_returns.sort_index()
returns = daily_returns / 1_000_000

# QuantStats 報表
qs.extend_pandas()
returns = returns.asfreq('D').fillna(0)
qs.reports.basic(returns)
plt.show()

# 畫出指定範圍圖
#cerebro.plot()



[MACD LONG ENTRY] Buy at 36035.43 on 2021-01-13
[MACD LONG FILLED] Buy at 36035.68, size: 1
[MACD LONG ENTRY] Buy at 36448.42 on 2021-01-18
[MACD LONG FILLED] Buy at 36448.67, size: 1
[MACD LONG ENTRY] Buy at 32986.34 on 2021-01-22
[MACD LONG FILLED] Buy at 32990.08, size: 1
[MACD LONG ENTRY] Buy at 32474.14 on 2021-01-26
[MACD LONG FILLED] Buy at 32474.81, size: 1
[MACD LONG ENTRY] Buy at 31937.9 on 2021-01-28
[MACD LONG FILLED] Buy at 31938.15, size: 1
[MACD LONG ENTRY] Buy at 34821.61 on 2021-02-02
[MACD LONG FILLED] Buy at 34821.86, size: 1
[MACD LONG ENTRY] Buy at 38006.64 on 2021-02-05
[MACD LONG FILLED] Buy at 38006.89, size: 1
[MACD LONG ENTRY] Buy at 39441.31 on 2021-02-06
[MACD LONG FILLED] Buy at 39441.56, size: 1
[MACD LONG ENTRY] Buy at 43785.81 on 2021-02-08
[MACD LONG FILLED] Buy at 43786.06, size: 1
[MACD LONG ENTRY] Buy at 51181.48 on 2021-02-17
[MACD LONG FILLED] Buy at 51188.71, size: 1
[MACD LONG ENTRY] Buy at 53944.7 on 2021-02-19
[MACD LONG FILLED] Buy at 53944.95

KeyError: 'datetime'